In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
import glob
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
import os

In [2]:
# models = ["FBCSP_SVM", "DeepConvNet", "EEGNet", "SpectralSpatialCNN", "MIN2Net_original", "MixNet"]
models = ["DeepConvNet", "EEGNet"] # Define list of your trained models 
datasets  = ['HighGamma'] # Define list of datasets used 
task = "subject_dependent"
result_path = "logs" # Path to read
save_path = "logs/" # Path to save

In [3]:
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [4]:
def load_y(path, subject):
    y_true, y_pred = [], []
    for subject in range(1,n_subjects+1):
        data = np.load(os.path.join(path, 'S{:03d}_Y_results.npz'.format(subject)), allow_pickle=True)
        y_true.append(data['y_true'])
        y_pred.append(data['y_pred'])
    return np.array(y_true), np.array(y_pred)

def eval_metric(y_true, y_pred, average='macro'):
    f1, acc, pre, rec = [], [], [], []
    for y_t, y_p in zip(y_true, y_pred):
        f1.append(f1_score(y_t, y_p, average=average))
        acc.append(accuracy_score(y_t, y_p))
    f1 = np.array(f1)
    acc = np.array(acc)
    return acc, f1

In [5]:
frames = []
for dataset in datasets:
    df_list_acc = []
    df_list_f1 = []
    for model in models:
        results = []
        index = []
        log_path = glob.glob('{}/{}/{}_2_classes_{}*/'.format(result_path, model, task, dataset))[0]
        print('The logs path being used now is: ', log_path)
        for subject in range(0,55):
            try:

                data = np.load(log_path+'S{:03d}_prediction_results.npy'.format(subject), allow_pickle=True)
                y_true = [data[i]['y_true'] for i in range(len(data))]
                y_pred = [data[j]['y_pred'] for j in range(len(data))] 
                acc, f1= eval_metric(y_true, y_pred, average='macro')
                results.append([acc.T, f1.T])

            except:
                pass

        results = np.array(results)
        dfx_acc = pd.DataFrame(results[:,0].reshape(-1,1), columns = ['accuracy'])
        dfx_f1 = pd.DataFrame(results[:,1].reshape(-1,1), columns = ['f1-score'])
        df_list_acc.append(dfx_acc)
        df_list_f1.append(dfx_f1)
    dfx_cat_acc = pd.concat(df_list_acc, axis=1, join='outer')
    dfx_cat_f1 = pd.concat(df_list_f1, axis=1, join='outer')
    dfx_cat = pd.concat([dfx_cat_acc, dfx_cat_f1], axis=1, join='outer')
    
    # If you wanna save all results from all folds, please remove the comment (#) from the below code
    # dfx_cat.to_csv(save_path+'merge_'+dataset+'_'+task+'.csv', index=False)
    if len(datasets) == 1 and len(models) == 1:
        acc_s  = zip([dfx_cat['accuracy'].mean()*100], [dfx_cat['accuracy'].std()*100])
        f1_s = zip([dfx_cat['f1-score'].mean()*100], [dfx_cat['f1-score'].std()*100])
    else:
        acc_s  = zip(dfx_cat['accuracy'].mean()*100, dfx_cat['accuracy'].std()*100)
        f1_s = zip(dfx_cat['f1-score'].mean()*100, dfx_cat['f1-score'].std()*100)
    
    acc_text = ['{:0.2f} \u00B1 {:0.2f}'.format(a, sd) for a, sd in acc_s]
    f1_text = ['{:0.2f} \u00B1 {:0.2f}'.format(a, sd) for a, sd in f1_s]
    
    text = np.array([[dataset]*len(models), models, acc_text, f1_text]).T
    
    frames.append(pd.DataFrame(text, columns = ['Dataset', 'Models', 'Accuracy', 'F1-score']))
    
display(pd.concat(frames))       

The logs path being used now is:  logs/DeepConvNet/subject_dependent_2_classes_HighGamma/
The logs path being used now is:  logs/EEGNet/subject_dependent_2_classes_HighGamma/


,Dataset,Models,Accuracy,F1-score
0,HighGamma,DeepConvNet,71.75 ± 20.04,70.08 ± 21.81
1,HighGamma,EEGNet,70.80 ± 19.50,69.62 ± 20.84
